In [15]:
from skimage.feature import graycomatrix, graycoprops
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from skimage.feature import local_binary_pattern
from scipy.stats import chisquare
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.svm import SVC
from PIL import Image
import pandas as pd
import numpy as np
import cv2
import os

newSize = 128 # Dimensão 128, 64, 32, 16, 8
dataDir = './data'

In [146]:
def doLBP(ImageArray):
    lbp = local_binary_pattern(ImageArray, 3, 8 * 3)
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, density=True, bins=n_bins, range=(0, n_bins))

    return hist.tolist()

def doGLCM(ImageArray, angles_range):
    glcmMatrix = np.empty((0,len(angles_range)),float)
    glcm = graycomatrix(ImageArray, distances=range(0,100,2), angles=angles_range, levels=256, symmetric=True, normed=True)
    
    glcmContrast = graycoprops(glcm, 'contrast')
    glcmDissimilarity = graycoprops(glcm, 'dissimilarity')
    glcmHomogeneity = graycoprops(glcm, 'homogeneity')
    glcmASM = graycoprops(glcm, 'ASM')
    glcmEnergy = graycoprops(glcm, 'energy')
    glcmCorrelation = graycoprops(glcm, 'correlation')

    glcmMatrix = np.append(glcmMatrix, glcmContrast, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmDissimilarity, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmHomogeneity, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmASM, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmEnergy, axis=0)
    glcmMatrix = np.append(glcmMatrix, glcmCorrelation, axis=0)

    return glcmMatrix.flatten().tolist()

def doPCA(dataFrame, newSize=128):
    y = dataFrame[0]
    X = dataFrame.drop(0, axis=1).values

    pca = PCA(n_components=newSize)
    pca.fit(X)

    XPCA = pca.transform(X)

    dataFramePCA = pd.DataFrame(XPCA)
    dataFramePCA['Classe'] = y
    dataFramePCA = dataFramePCA[['Classe'] + dataFramePCA.columns[:-1].tolist()]
    
    return dataFramePCA

def doPCAArray(array, newSize=128):
    pca = PCA(n_components= newSize if len(array) > newSize else int(len(array)/2))
    pca.fit(array)
    
    return pca.transform(array).tolist()

def doKnn(X_train, y_train, X_teste, y_teste, MAX_K=30):
    range_k = np.arange(1, MAX_K, 1)
    scores = {}

    for k in range_k:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_teste)

        scores[k] = metrics.accuracy_score(y_teste, y_pred)
    
    return scores, range_k

def doCVM(X_train, y_train, X_teste, y_teste, c=1.0, gamma='scale'):
    svm = SVC(C=c, gamma=gamma)
    svm.fit(X_train, y_train)

    return svm.score(X_teste, y_teste)
    
# def doSTIF(ImageArray):
#     sift = SIFT()
#     sift.detect_and_extract(ImageArray)
    
#     return sift.descriptors.flatten().tolist()
global stif
sift = cv2.SIFT_create()

def doSTIF(ImageArray):
    global sift
    keypoints_1, _ = sift.detectAndCompute(ImageArray, None)

    return [(v.pt[0], v.pt[1]) for v in keypoints_1]

def doSTIF2(ImageArray):
    global sift
    keypoints_1, _ = sift.detectAndCompute(ImageArray, None)

    x = [v.pt[0] for v in keypoints_1]
    y = [v.pt[1] for v in keypoints_1]
    return x, y

def doChisquare(Image):
    return chisquare(Image)[0].tolist()

In [183]:
maxH = 2073
maxW = 2339
dataTeste = []
dataTreino = []

angle_step = np.pi/4
angle_end = np.pi
angles_range = np.arange(0, angle_end, angle_step)

def run(Image):

    # Redimensiona a imagem para 2339x2073
    ImageResize = Image.resize((maxW, maxH))

    # Converte a imagem para um array
    ImageArray = np.asarray(ImageResize.convert('L'))

    # Aplica o LBP
    lbp = doLBP(ImageArray)

    # Aplica o GLCM
    glcm = doGLCM(ImageArray, angles_range)

    # Aplica o STIF
    stifX, stifY = doSTIF2(ImageArray)

    # Aplica o Chisquare
    chisquare = doChisquare(ImageArray)

    return lbp, glcm, stifX, stifY, chisquare

def runPartition(Image, nPartitions):
    
    lbp = []
    glcm = []
    stifX = []
    stifY = []
    chisquare = []

    # Redimensiona a imagem para 2339x2073
    ImageResize = Image.resize((maxW, maxH))

    # Converte a imagem para um array
    ImageArray = np.asarray(ImageResize.convert('L'))

    tamPartition = len(ImageArray)/nPartitions
    posI = 0
    posF = tamPartition

    while posF <= len(ImageArray):

        # Aplica o LBP
        lbp += doLBP(ImageArray[int(posI):int(posF)])

        # Aplica o GLCM
        glcm += doGLCM(ImageArray[int(posI):int(posF)], angles_range)

        # Aplica o STIF
        stifXValue, stifYValue = doSTIF2(ImageArray[int(posI):int(posF)])
        stifX += stifXValue
        stifY += stifYValue

        # Aplica o Chisquare
        chisquare += doChisquare(ImageArray[int(posI):int(posF)])

        posI += tamPartition
        posF += tamPartition
        
    return lbp, glcm, stifX, stifY, chisquare

def returnData(Image, partition, nPartitions):
    return run(Image) if not partition else runPartition(Image, nPartitions)

def fillArrayWithZeros(array, size):
    newArray = []

    for values in array:
        newArray.append(values + np.zeros(size - len(values)))

    return newArray

def fillArrayWithZeros2(array, size):
    newArray = []

    for values in array:
        newArray.append(values + [0] * (size - len(values)))

    return newArray

'''
    partition: True - Particiona a imagem em quadrantes
    nPartitions: Quantidade de quadrantes
    limit: Quantidade de classes a serem processadas
'''

def generateData(partition=False, nPartitions=2, limit=0, doPCAFeatures=False):

    files = os.listdir(dataDir)
    limit = len(files) if limit == 0 else limit

    classe = []
    lbpListTreino,  glcmListTreino, stifListTreinoX, stifListTreinoY, chisquareListTreino = [], [], [], [], []
    lbpListTeste,  glcmListTeste, stifListTesteX, stifListTesteY, chisquareListTeste = [], [], [], [], []

    maiorStifTreino = []
    maiorChisquareTreino = []
    maiorStifTeste = []
    maiorChisquareTeste = []

    for i, dir in enumerate(files[:limit]):
        print(f'Run image n° {i}...')

        # Lista todas as imagens do direttório
        images = os.listdir(f'{dataDir}/{dir}/')

        # Abrir as imagens de treino e teste
        ImageDefaultTreino = Image.open(f'{dataDir}/{dir}/{images[0]}')
        ImageDefaultTeste = Image.open(f'{dataDir}/{dir}/{images[1]}')

        lbpList, glcmList, stifListX, stifListY, chisquareList = returnData(ImageDefaultTreino, partition, nPartitions)

        lbpListTreino.append(lbpList)
        glcmListTreino.append(glcmList)
        stifListTreinoX.append(stifListX)
        stifListTreinoY.append(stifListY)
        chisquareListTreino.append(chisquareList)
        classe.append([i])

        if len(stifListX) > len(maiorStifTreino):
            maiorStifTreino = stifListX

        if len(chisquareList) > len(maiorChisquareTreino):
            maiorChisquareTreino = chisquareList
            
        lbpList, glcmList, stifListX, stifListY, chisquareList = returnData(ImageDefaultTeste, partition, nPartitions)

        lbpListTeste.append(lbpList)
        glcmListTeste.append(glcmList)
        stifListTesteX.append(stifListX)
        stifListTesteY.append(stifListY)
        chisquareListTeste.append(chisquareList)

        if len(stifListX) > len(maiorStifTeste):
            maiorStifTeste = stifListX

        if len(chisquareList) > len(maiorChisquareTeste):
            maiorChisquareTeste = chisquareList

    # treino
    stifListTreinoX = fillArrayWithZeros2(stifListTreinoX, len(maiorStifTreino))
    stifListTreinoY = fillArrayWithZeros2(stifListTreinoY, len(maiorStifTreino))
    chisquareListTreino = fillArrayWithZeros2(chisquareListTreino, len(maiorChisquareTreino))
    
    if doPCAFeatures:
        glcmListTreino = doPCAArray(glcmListTreino)
        stifListTreinoX = doPCAArray(stifListTreinoX)
        stifListTreinoY = doPCAArray(stifListTreinoY)
        chisquareListTreino = doPCAArray(chisquareListTreino)
        

    # teste
    stifListTesteX = fillArrayWithZeros2(stifListTesteX, len(maiorStifTeste))
    stifListTesteY = fillArrayWithZeros2(stifListTesteY, len(maiorStifTeste))
    chisquareListTeste = fillArrayWithZeros2(chisquareListTeste, len(maiorChisquareTeste))
    
    if doPCAFeatures:
        glcmListTeste = doPCAArray(glcmListTeste)
        stifListTesteX = doPCAArray(stifListTreinoX)
        stifListTesteY = doPCAArray(stifListTesteY)
        chisquareListTeste = doPCAArray(chisquareListTeste)

    dataTreino = [a + b + c + d + e + f for a, b, c, d, e, f in zip(classe, lbpListTreino, glcmListTreino, stifListTreinoX, stifListTreinoY, chisquareListTreino)]
    dataTeste = [a + b + c + d + e + f for a, b, c, d, e, f in zip(classe, lbpListTeste, glcmListTeste, stifListTesteX, stifListTesteY, chisquareListTeste)]

    return dataTreino, dataTeste

In [201]:
doPCAFeatures = True
dataTreino, dataTeste = generateData(False, 3, 0, doPCAFeatures)

Run image n° 0...
Run image n° 1...
Run image n° 2...
Run image n° 3...
Run image n° 4...
Run image n° 5...
Run image n° 6...
Run image n° 7...
Run image n° 8...
Run image n° 9...
Run image n° 10...
Run image n° 11...
Run image n° 12...
Run image n° 13...
Run image n° 14...
Run image n° 15...
Run image n° 16...
Run image n° 17...
Run image n° 18...
Run image n° 19...
Run image n° 20...
Run image n° 21...
Run image n° 22...
Run image n° 23...
Run image n° 24...
Run image n° 25...
Run image n° 26...
Run image n° 27...
Run image n° 28...
Run image n° 29...
Run image n° 30...
Run image n° 31...
Run image n° 32...
Run image n° 33...
Run image n° 34...
Run image n° 35...
Run image n° 36...
Run image n° 37...
Run image n° 38...
Run image n° 39...
Run image n° 40...
Run image n° 41...
Run image n° 42...
Run image n° 43...
Run image n° 44...
Run image n° 45...
Run image n° 46...
Run image n° 47...
Run image n° 48...
Run image n° 49...
Run image n° 50...
Run image n° 51...
Run image n° 52...
Run

In [202]:
dataFrameTreino = pd.DataFrame(dataTreino)
dataFrameTeste = pd.DataFrame(dataTeste)

# dataFrameTreino = dataFrameTreino.fillna(0)
# dataFrameTeste = dataFrameTeste.fillna(0)

dataFrameTreino.to_csv('data_treino.csv', index=False)
dataFrameTeste.to_csv('data_teste.csv', index=False)

In [203]:
if not doPCAFeatures:
    newSize = 128 # Dimensão 128, 64, 32, 16, 8
    dataFrameTreino = doPCA(dataFrameTreino, newSize if len(dataFrameTreino) > newSize else len(dataFrameTreino))
    dataFrameTeste = doPCA(dataFrameTeste, newSize if len(dataFrameTeste) > newSize else len(dataFrameTeste))
else:
    dataFrameTreino.rename(columns={0: 'Classe'}, inplace=True)
    dataFrameTeste.rename(columns={0: 'Classe'}, inplace=True)

In [204]:
scores, range_k = doKnn(dataFrameTreino.drop(['Classe'], axis=1).values, dataFrameTreino['Classe'].values, dataFrameTeste.drop(['Classe'], axis=1).values, dataFrameTeste['Classe'].values, 30 if len(dataFrameTeste) > 30 else len(dataFrameTeste)-1)
scores

{1: 0.1461794019933555,
 2: 0.09302325581395349,
 3: 0.06976744186046512,
 4: 0.06312292358803986,
 5: 0.0664451827242525,
 6: 0.0664451827242525,
 7: 0.059800664451827246,
 8: 0.05647840531561462,
 9: 0.046511627906976744,
 10: 0.04318936877076412,
 11: 0.04318936877076412,
 12: 0.04318936877076412,
 13: 0.046511627906976744,
 14: 0.036544850498338874,
 15: 0.03986710963455149,
 16: 0.03986710963455149,
 17: 0.03986710963455149,
 18: 0.036544850498338874,
 19: 0.03322259136212625,
 20: 0.036544850498338874,
 21: 0.03322259136212625,
 22: 0.03322259136212625,
 23: 0.03322259136212625,
 24: 0.03322259136212625,
 25: 0.03322259136212625,
 26: 0.03322259136212625,
 27: 0.029900332225913623,
 28: 0.029900332225913623,
 29: 0.029900332225913623}

In [205]:
svm = doCVM(dataFrameTreino.drop(['Classe'], axis=1).values, dataFrameTreino['Classe'].values, dataFrameTeste.drop(['Classe'], axis=1).values, dataFrameTeste['Classe'].values)
svm

0.1461794019933555

In [206]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(dataFrameTreino.drop(['Classe'], axis=1).values, dataFrameTreino['Classe'].values)
predictions = rf.predict(dataFrameTeste.drop(['Classe'], axis=1).values)
accuracy = 100 - np.mean(100 * (abs(predictions - dataFrameTeste['Classe'].values) / (dataFrameTeste['Classe'].values)))
accuracy

RandomForestRegressor(n_estimators=1000, random_state=42)